## Prepare libraries and data

In [1]:
import numpy as np
import pandas as pd
import keras
import keras_nlp
from sklearn.model_selection import train_test_split

from keras import Input, Model
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Dense, Dropout, LeakyReLU, UnitNormalization, Reshape
from keras.optimizers import RMSprop
from keras.losses import CategoricalCrossentropy
from keras.metrics import F1Score

2024-07-29 14:45:29.323288: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 14:45:29.323579: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 14:45:29.325814: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 14:45:29.354159: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 14:45:29.896163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
import tensorflow as tf

In [3]:
preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset("roberta_base_en")
backbone = keras_nlp.models.RobertaBackbone.from_preset("roberta_base_en")
backbone.trainable = False

inputs = Input(shape=(1,), dtype="string")
preprocess = preprocessor(inputs)
embed = backbone(preprocess)

In [4]:
encoder = Model(inputs=inputs, outputs=embed)

In [5]:
cnn = keras.saving.load_model('roberta_cnn.keras')

In [6]:
embed_cnn = cnn(embed)

In [7]:
full_model = Model(inputs=inputs, outputs=embed_cnn)

In [8]:
full_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ roberta_preprocess… │ [(None, 512),     │          0 │ input_layer[0][0] │
│ (RobertaPreprocess… │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ roberta_backbone    │ (None, 512, 768)  │ 124,052,7… │ roberta_preproce… │
│ (RobertaBackbone)   │                   │            │ roberta_preproce… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 6)         │    321,958 │ roberta_backbone… │
│ (Functional)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,374,694 (474.45 MB)

 Trainable params: 321,958 (1.23 MB)

 Non-trainable params: 124,052,736 (473.22 MB)

# Generate LIME explanations

In [9]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [10]:
levels = ['Analysis', 'Application', 'Comprehension', 'Evaluation', 'Knowledge', 'Synthesis']

In [11]:
explainer = LimeTextExplainer(class_names=levels)

In [12]:
examples = pd.read_csv('explanations.csv', sep=';')

In [13]:
def predictor(texts):
    encoding = encoder.predict(tf.convert_to_tensor([[text] for text in texts]), verbose=0)
    probs = cnn.predict(encoding, verbose=0)
    return probs

In [36]:
def explanation(df, index, labels):
    question = df.Question[index]
    exp = explainer.explain_instance(question, predictor, labels=labels, num_features=10, num_samples=100)
    return exp

In [ ]:
with open('exps.txt', 'w') as exps:
    for idx in range(50):
        level = examples.BT_level[idx]
        li = levels.index(level)
        exp = explanation(examples, idx, list(range(6)))
        exps.write(str(exp.as_list(label=li))+'\n')